# Notebook to investigate the `som3.py` script

## Running the original code

In [1]:
import random
import sys
import xarray as xr
import logging
import numpy as np
import numpy.random

In [2]:
# Progress class from original code
class Progress(object):

    def __init__(self,label,silent=False):
        self.label = label
        self.last_progress_frac = None
        self.silent = silent

    def report(self,msg,progress_frac):
        if self.silent:
            return
        if self.last_progress_frac is None or (progress_frac - self.last_progress_frac) >= 0.01:
            self.last_progress_frac = progress_frac
            i = int(100*progress_frac)
            if i > 100:
                i = 100
            si = i // 2
            sys.stdout.write("\r%s %s %-05s %s" % (self.label,msg,str(i)+"%","#"*si))
            sys.stdout.flush()

    def complete(self,msg):
        if self.silent:
            return
        sys.stdout.write("\n%s %s\n" % (self.label,msg))
        sys.stdout.flush()

In [3]:
def find_bmu(instances, weights):
    sqdiffs = (instances[:, :, None] - np.transpose(weights)) ** 2
    sumsqdiffs = sqdiffs.sum(axis=1)
    return sumsqdiffs.argmin(axis=1)

In [4]:
def train_batch(instances, weights, learn_rate, neighbourhood_lookup):
    # winners(#instances) holds the index of the closest weight for each instance
    winners = find_bmu(instances, weights)
    # now find the neighbours of each winner that are also activated by each instance
    # nhoods(#activations,2) holds the instance index and the weight index for each activation
    nwinners = neighbourhood_lookup[winners, :]
    nhoods = np.argwhere(nwinners)

    # get the indices
    weight_indices = nhoods[:, 1]
    instance_indices = nhoods[:, 0]
    fractions = nwinners[instance_indices, weight_indices]
    # print(fractions.shape,np.min(fractions),np.max(fractions))

    # get the updates
    updates = -learn_rate * fractions[:, None] * (weights[weight_indices, :] - instances[instance_indices])

    # aggregate the updates for each weight
    numerator = np.zeros(shape=weights.shape)
    np.add.at(numerator, weight_indices, updates)
    denominator = np.zeros(shape=weights.shape[:1])[:, None]
    np.add.at(denominator, weight_indices, 1)
    denominator = np.where(numerator == 0, 1, denominator)  # fix annoying divide by zero warning
    weight_updates = numerator / denominator

    # update the weights
    weights += weight_updates

In [5]:
def compute_scores(instances, weights, gridwidth, minibatch_size):
    index = 0
    nr_instances = instances.shape[0]
    batch_size = nr_instances if not minibatch_size else minibatch_size
    bmus = np.zeros(shape=(nr_instances,), dtype=int)
    while index < nr_instances:
        last_index = min(index + batch_size, nr_instances)
        bmus[index:last_index] = find_bmu(instances[index:last_index], weights)
        index += batch_size
    scores = np.vstack([bmus % gridwidth, bmus // gridwidth])
    return np.transpose(scores)

In [6]:
class SelfOrganisingMap(object):
    """
    Train Self Organising Map (SOM) with cells arranged in a 2-dimensional rectangular layout

    Parameters
    ----------
    iters : int
        the number of training iterations to use when training the SOM
    gridwidth : int
        number of cells across the grid
    gridheight : int
        number of cells down the grid
    initial_neighbourhood : int
        the initial neighbourhood size

    Keyword Parameters
    ------------------
    verbose : bool
        whether to print progress messages
    seed : int
        random seed - set to produce repeatable results
    """

    def __init__(self, gridwidth, gridheight, iters, initial_neighbourhood=None, verbose=False, seed=None,
                 minibatch_size=None):
        self.gridheight = gridheight
        self.gridwidth = gridwidth
        self.nr_outputs = self.gridwidth * self.gridheight
        self.iters = iters
        self.minibatch_size = minibatch_size

        self.initial_neighbourhood = initial_neighbourhood if initial_neighbourhood else int(gridsize / 3)
        self.verbose = verbose
        self.seed = seed
        self.rng = random.Random()
        if seed:
            self.rng.seed(seed)

        self.learn_rate_initial = 0.01
        self.learn_rate_final = 0.001
        self.neighbourhood_lookup = np.zeros(shape=(self.initial_neighbourhood + 1, self.nr_outputs, self.nr_outputs))

        # for each neighbourhood size 0,1,...initial_neighbourhood
        # build a lookup table where the fraction at neighbourhood_lookup[n,o1,o2]
        # indicates if (and how much) weight at index o2 is a neighbour of the weight at index o1 in neighbourhood size n
        # use 1 and 0 for a binary mask, or between -1.0 and 1.0 for a varying mask
        for neighbourhood in range(0, self.initial_neighbourhood + 1):
            nsq = neighbourhood ** 2
            for i in range(self.nr_outputs):
                ix, iy = self.coords(i)
                for j in range(self.nr_outputs):
                    jx, jy = self.coords(j)
                    sqdist = (jx - ix) ** 2 + (jy - iy) ** 2
                    self.neighbourhood_lookup[neighbourhood, i, j] = 1 if sqdist <= nsq else 0

    def get_weights(self, outputIndex):
        return self.weights[:, outputIndex].tolist()

    def fit_transform(self, original_instances):

        # mask out instances containing NaNs and remove them
        instance_mask = ~np.any(np.isnan(original_instances), axis=1)
        nr_original_instances = original_instances.shape[0]
        valid_instances = original_instances[instance_mask, :]

        # randomly re-shuffle the remaining instances.
        # TODO consider reshuffling after every iteration
        instances = np.copy(valid_instances)
        rng = np.random.default_rng(seed=self.seed)
        rng.shuffle(instances)

        nr_inputs = instances.shape[1]
        nr_instances = instances.shape[0]

        weights = np.zeros((self.nr_outputs, nr_inputs))
        for output_idx in range(0, self.nr_outputs):
            weights[output_idx, :] = instances[self.rng.choice(range(0, nr_instances)), :]

        p = Progress("SOM", silent=not self.verbose)
        progress_frac = 0.0
        p.report("Starting", progress_frac)

        for iteration in range(self.iters):
            # reduce the learning rate and neighbourhood size linearly as training progresses
            learn_rate = self.learn_rate_initial - (self.learn_rate_initial - self.learn_rate_final) * (
                        (iteration + 1) / self.iters)
            neighbour_limit = round(self.initial_neighbourhood * (1 - (iteration + 1) / self.iters))
            neighbourhood_mask = self.neighbourhood_lookup[neighbour_limit, :, :]
            batch_size = nr_instances if not minibatch_size else minibatch_size

            index = 0
            while index < nr_instances:
                last_index = min(index + batch_size, nr_instances)
                train_batch(instances[index:last_index, :], weights, learn_rate, neighbourhood_mask)
                index += batch_size

            progress_frac = iteration / self.iters
            p.report("Training neighbourhood=%d" % (neighbour_limit), progress_frac)

        p.complete("SOM Training Complete")

        # compute final scores from the trained weights
        valid_scores = compute_scores(valid_instances, weights, self.gridwidth, self.minibatch_size)

        # restore the results into the same order as the input array
        scores = np.zeros(shape=(nr_original_instances, 2))
        scores[:, :] = np.nan
        scores[instance_mask, :] = valid_scores
        return scores

    def coords(self, output):
        return (output % self.gridwidth, output // self.gridwidth)

    def get_output(self, x, y):
        return x + (y * self.gridwidth)

In [7]:
gridsize = 16
gridheight = 16
iters = 10
minibatch_size = 1000  # the max number of instances passed in each call to train_batch

logging.basicConfig(level=logging.DEBUG)

da = xr.open_dataset("data/sla_c3s_clim.nc")[
    "sla_c3s"]  # sea level anomalies averaged by month-of-year, lat and lon cell

stack_dims = ("lat", "lon")
stack_sizes = (da.shape[1], da.shape[2])

# each (lat,lon) position becomes an independent case
# flatten lat and lon dimensions and transpose to arrange by (ncases, time)
# where ncases = nlat*nlon
instances = da.stack(case=stack_dims).transpose("case", "month").values

# run SOM to reduce time dimension from 12 to 2
s = SelfOrganisingMap(gridsize, gridsize, iters, seed=1, verbose=True, minibatch_size=minibatch_size)

import time

start_time = time.time()
scores = s.fit_transform(instances)
end_time = time.time()
print("Elapsed time: %d seconds" % (int(end_time - start_time)))

# restore lat/lon dimensions and output
a = scores.reshape(stack_sizes + (2,))
new_dims = stack_dims + ("som_axis",)
output = xr.DataArray(data=a, dims=new_dims, name="monthly_sla_som")
output.to_netcdf("som.nc")

SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
Elapsed time: 44 seconds


This already runs quite fast on the CPU using the new batch approach. 

## Using CuPy as drop in replacement for NumPy

In [8]:
import cupy as np

def find_bmu(instances, weights):
    sqdiffs = (instances[:, :, None] - np.transpose(weights)) ** 2
    sumsqdiffs = sqdiffs.sum(axis=1)
    return sumsqdiffs.argmin(axis=1)

def train_batch(instances, weights, learn_rate, neighbourhood_lookup):
    # winners(#instances) holds the index of the closest weight for each instance
    winners = find_bmu(instances, weights)
    # now find the neighbours of each winner that are also activated by each instance
    # nhoods(#activations,2) holds the instance index and the weight index for each activation
    nwinners = neighbourhood_lookup[winners, :]
    nhoods = np.argwhere(nwinners)

    # get the indices
    weight_indices = nhoods[:, 1]
    instance_indices = nhoods[:, 0]
    fractions = nwinners[instance_indices, weight_indices]
    # print(fractions.shape,np.min(fractions),np.max(fractions))

    # get the updates
    updates = -learn_rate * fractions[:, None] * (weights[weight_indices, :] - instances[instance_indices])

    # aggregate the updates for each weight
    numerator = np.zeros(shape=weights.shape)
    #np.add.at(numerator, weight_indices, updates)     #### change required here for cupy - no cp.add.at
    numerator[weight_indices] = numerator[weight_indices] + updates
    denominator = np.zeros(shape=weights.shape[:1])[:, None]
    #np.add.at(denominator, weight_indices, 1)     #### change required here for cupy - no cp.add.at
    denominator[weight_indices] = denominator[weight_indices] + 1
    denominator = np.where(numerator == 0, 1, denominator)  # fix annoying divide by zero warning
    weight_updates = numerator / denominator

    # update the weights
    weights += weight_updates
    
def compute_scores(instances, weights, gridwidth, minibatch_size):
    index = 0
    nr_instances = instances.shape[0]
    batch_size = nr_instances if not minibatch_size else minibatch_size
    bmus = np.zeros(shape=(nr_instances,), dtype=int)
    while index < nr_instances:
        last_index = min(index + batch_size, nr_instances)
        bmus[index:last_index] = find_bmu(instances[index:last_index], weights)
        index += batch_size
    scores = np.vstack([bmus % gridwidth, bmus // gridwidth])
    return np.transpose(scores)

class SelfOrganisingMap(object):
    """
    Train Self Organising Map (SOM) with cells arranged in a 2-dimensional rectangular layout

    Parameters
    ----------
    iters : int
        the number of training iterations to use when training the SOM
    gridwidth : int
        number of cells across the grid
    gridheight : int
        number of cells down the grid
    initial_neighbourhood : int
        the initial neighbourhood size

    Keyword Parameters
    ------------------
    verbose : bool
        whether to print progress messages
    seed : int
        random seed - set to produce repeatable results
    """

    def __init__(self, gridwidth, gridheight, iters, initial_neighbourhood=None, verbose=False, seed=None,
                 minibatch_size=None):
        self.gridheight = gridheight
        self.gridwidth = gridwidth
        self.nr_outputs = self.gridwidth * self.gridheight
        self.iters = iters
        self.minibatch_size = minibatch_size

        self.initial_neighbourhood = initial_neighbourhood if initial_neighbourhood else int(gridsize / 3)
        self.verbose = verbose
        self.seed = seed
        self.rng = random.Random()
        if seed:
            self.rng.seed(seed)

        self.learn_rate_initial = 0.01
        self.learn_rate_final = 0.001
        self.neighbourhood_lookup = np.zeros(shape=(self.initial_neighbourhood + 1, self.nr_outputs, self.nr_outputs))

        # for each neighbourhood size 0,1,...initial_neighbourhood
        # build a lookup table where the fraction at neighbourhood_lookup[n,o1,o2]
        # indicates if (and how much) weight at index o2 is a neighbour of the weight at index o1 in neighbourhood size n
        # use 1 and 0 for a binary mask, or between -1.0 and 1.0 for a varying mask
        for neighbourhood in range(0, self.initial_neighbourhood + 1):
            nsq = neighbourhood ** 2
            for i in range(self.nr_outputs):
                ix, iy = self.coords(i)
                for j in range(self.nr_outputs):
                    jx, jy = self.coords(j)
                    sqdist = (jx - ix) ** 2 + (jy - iy) ** 2
                    self.neighbourhood_lookup[neighbourhood, i, j] = 1 if sqdist <= nsq else 0

#     def get_weights(self, outputIndex):
#         return self.weights[:, outputIndex].tolist()

    def fit_transform(self, original_instances):

        # mask out instances containing NaNs and remove them
        instance_mask = ~np.any(np.isnan(original_instances), axis=1)
        nr_original_instances = original_instances.shape[0]
        valid_instances = original_instances[instance_mask, :]

        # randomly re-shuffle the remaining instances.
        # TODO consider reshuffling after every iteration
        instances = np.copy(valid_instances)
        #rng = np.random.default_rng(seed=self.seed)     #### change required here for cupy - no cp.random.default_rng
        #rng.shuffle(instances)
        np.random.shuffle(instances)  # will change every time

        nr_inputs = instances.shape[1]
        nr_instances = instances.shape[0]

        weights = np.zeros((self.nr_outputs, nr_inputs))
        for output_idx in range(0, self.nr_outputs):
            weights[output_idx, :] = instances[self.rng.choice(range(0, nr_instances)), :]

        p = Progress("SOM", silent=not self.verbose)
        progress_frac = 0.0
        p.report("Starting", progress_frac)

        for iteration in range(self.iters):
            # reduce the learning rate and neighbourhood size linearly as training progresses
            learn_rate = self.learn_rate_initial - (self.learn_rate_initial - self.learn_rate_final) * (
                        (iteration + 1) / self.iters)
            neighbour_limit = round(self.initial_neighbourhood * (1 - (iteration + 1) / self.iters))
            neighbourhood_mask = self.neighbourhood_lookup[neighbour_limit, :, :]
            batch_size = nr_instances if not minibatch_size else minibatch_size

            index = 0
            while index < nr_instances:
                last_index = min(index + batch_size, nr_instances)
                train_batch(instances[index:last_index, :], weights, learn_rate, neighbourhood_mask)
                index += batch_size

            progress_frac = iteration / self.iters
            p.report("Training neighbourhood=%d" % (neighbour_limit), progress_frac)

        p.complete("SOM Training Complete")

        # compute final scores from the trained weights
        valid_scores = compute_scores(valid_instances, weights, self.gridwidth, self.minibatch_size)

        # restore the results into the same order as the input array
        scores = np.zeros(shape=(nr_original_instances, 2))
        scores[:, :] = np.nan
        scores[instance_mask, :] = valid_scores
        return scores

    def coords(self, output):
        return (output % self.gridwidth, output // self.gridwidth)

#     def get_output(self, x, y):
#         return x + (y * self.gridwidth)

In [9]:
gridsize = 16
gridheight = 16
iters = 10
minibatch_size = 1000  # the max number of instances passed in each call to train_batch

logging.basicConfig(level=logging.DEBUG)

da = xr.open_dataset("data/sla_c3s_clim.nc")[
    "sla_c3s"]  # sea level anomalies averaged by month-of-year, lat and lon cell

stack_dims = ("lat", "lon")
stack_sizes = (da.shape[1], da.shape[2])

# each (lat,lon) position becomes an independent case
# flatten lat and lon dimensions and transpose to arrange by (ncases, time)
# where ncases = nlat*nlon
instances = da.stack(case=stack_dims).transpose("case", "month").values
instances = np.asarray(instances)    ### change required here - need to make instances a cupy array

# run SOM to reduce time dimension from 12 to 2
s = SelfOrganisingMap(gridsize, gridsize, iters, seed=1, verbose=True, minibatch_size=minibatch_size)

import time

start_time = time.time()
scores = s.fit_transform(instances)
end_time = time.time()
print("Elapsed time: %d seconds" % (int(end_time - start_time)))

SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
Elapsed time: 1 seconds


See a good speed up with very minimal changes:
* Needed to convert instances array to cp.ndarray before passing to `fit_transform()`,
* needed to swap out the code that used the `np.add.at` method,
* needed to swap out the code that used the `np.random.default_rng` method - this means the results change every time.

In [10]:
gridsize = 16
gridheight = 16
iters = 10
minibatch_size = None#1000  # the max number of instances passed in each call to train_batch

logging.basicConfig(level=logging.DEBUG)

da = xr.open_dataset("data/sla_c3s_clim.nc")[
    "sla_c3s"]  # sea level anomalies averaged by month-of-year, lat and lon cell

stack_dims = ("lat", "lon")
stack_sizes = (da.shape[1], da.shape[2])

# each (lat,lon) position becomes an independent case
# flatten lat and lon dimensions and transpose to arrange by (ncases, time)
# where ncases = nlat*nlon
instances = da.stack(case=stack_dims).transpose("case", "month").values
instances = np.asarray(instances)

# run SOM to reduce time dimension from 12 to 2
s = SelfOrganisingMap(gridsize, gridsize, iters, seed=1, verbose=True, minibatch_size=minibatch_size)

import time

start_time = time.time()
scores = s.fit_transform(instances)
end_time = time.time()
print("Elapsed time: %d seconds" % (int(end_time - start_time)))

SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
Elapsed time: 0 seconds


Changing minibatch size to None above shows timing of 0 seconds. Let's check a few setting with more precise timings...

In [11]:
s = SelfOrganisingMap(gridsize, gridsize, iters, seed=1, verbose=True, minibatch_size=1000)
%timeit scores = s.fit_transform(instances)

SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
450 ms ± 4.12 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
s = SelfOrganisingMap(gridsize, gridsize, iters, seed=1, verbose=True, minibatch_size=None)
%timeit scores = s.fit_transform(instances)

SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
SOM Training neighbourhood=0 90%   #############################################
SOM SOM Training Complete
447 ms ± 2.99 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
# we can comment out the progress reporting though it doesn't affect timings

In [14]:
s = SelfOrganisingMap(gridsize, gridsize, iters, seed=1, verbose=True, minibatch_size=1000)
%timeit scores = s.fit_transform(instances)

464 ms ± 1.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
s = SelfOrganisingMap(gridsize, gridsize, iters, seed=1, verbose=True, minibatch_size=None)
%timeit scores = s.fit_transform(instances)

461 ms ± 3.08 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
# playing with minibatch_size shows little difference in timings, but if you make them small enough it slows things down a little...

In [17]:
s = SelfOrganisingMap(gridsize, gridsize, iters, seed=1, verbose=True, minibatch_size=50)
%timeit scores = s.fit_transform(instances)

675 ms ± 4.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Original code, swapping out np for cp, using reduction

Next check if we get any improvements using the reduction kernel class in bmu calculations.

In [14]:
import cupy as np

# def find_bmu(instances, weights):
#     print(instances.shape, weights.shape)
#     sqdiffs = (instances[:, :, None] - np.transpose(weights)) ** 2
#     sumsqdiffs = sqdiffs.sum(axis=1)
#     return sumsqdiffs.argmin(axis=1)

sqsum_kernel = np.ReductionKernel(
    'T x, T y',  # input params
    'T z',  # output params
    '(x - y) * (x - y)',  # map
    'a + b',  # reduce
    'z = a',  # post-reduction map
    '0',  # identity value
    'sqsum'  # kernel name
    )

def find_bmu(instances, weights):
    inarr = instances[:, :, None]
    sumsdiffs = sqsum_kernel(inarr, weights.T, axis=1)
    return np.argmin(sumsdiffs, axis=1)  


def train_batch(instances, weights, learn_rate, neighbourhood_lookup):
    # winners(#instances) holds the index of the closest weight for each instance
    winners = find_bmu(instances, weights)
    # now find the neighbours of each winner that are also activated by each instance
    # nhoods(#activations,2) holds the instance index and the weight index for each activation
    nwinners = neighbourhood_lookup[winners, :]
    nhoods = np.argwhere(nwinners)

    # get the indices
    weight_indices = nhoods[:, 1]
    instance_indices = nhoods[:, 0]
    fractions = nwinners[instance_indices, weight_indices]
    # print(fractions.shape,np.min(fractions),np.max(fractions))

    # get the updates
    updates = -learn_rate * fractions[:, None] * (weights[weight_indices, :] - instances[instance_indices])

    # aggregate the updates for each weight
    numerator = np.zeros(shape=weights.shape)
    numerator[weight_indices] = numerator[weight_indices] + updates
    denominator = np.zeros(shape=weights.shape[:1])[:, None]
    denominator[weight_indices] = denominator[weight_indices] + 1
    denominator = np.where(numerator == 0, 1, denominator)  # fix annoying divide by zero warning
    weight_updates = numerator / denominator

    # update the weights
    weights += weight_updates
    
def compute_scores(instances, weights, gridwidth, minibatch_size):
    index = 0
    nr_instances = instances.shape[0]
    batch_size = nr_instances if not minibatch_size else minibatch_size
    bmus = np.zeros(shape=(nr_instances,), dtype=int)
    while index < nr_instances:
        last_index = min(index + batch_size, nr_instances)
        bmus[index:last_index] = find_bmu(instances[index:last_index], weights)
        index += batch_size
    scores = np.vstack([bmus % gridwidth, bmus // gridwidth])
    return np.transpose(scores)

class SelfOrganisingMap(object):
    """
    Train Self Organising Map (SOM) with cells arranged in a 2-dimensional rectangular layout

    Parameters
    ----------
    iters : int
        the number of training iterations to use when training the SOM
    gridwidth : int
        number of cells across the grid
    gridheight : int
        number of cells down the grid
    initial_neighbourhood : int
        the initial neighbourhood size

    Keyword Parameters
    ------------------
    verbose : bool
        whether to print progress messages
    seed : int
        random seed - set to produce repeatable results
    """

    def __init__(self, gridwidth, gridheight, iters, initial_neighbourhood=None, verbose=False, seed=None,
                 minibatch_size=None):
        self.gridheight = gridheight
        self.gridwidth = gridwidth
        self.nr_outputs = self.gridwidth * self.gridheight
        self.iters = iters
        self.minibatch_size = minibatch_size

        self.initial_neighbourhood = initial_neighbourhood if initial_neighbourhood else int(gridsize / 3)
        self.verbose = verbose
        self.seed = seed
        self.rng = random.Random()
        if seed:
            self.rng.seed(seed)

        self.learn_rate_initial = 0.01
        self.learn_rate_final = 0.001
        self.neighbourhood_lookup = np.zeros(shape=(self.initial_neighbourhood + 1, self.nr_outputs, self.nr_outputs))

        # for each neighbourhood size 0,1,...initial_neighbourhood
        # build a lookup table where the fraction at neighbourhood_lookup[n,o1,o2]
        # indicates if (and how much) weight at index o2 is a neighbour of the weight at index o1 in neighbourhood size n
        # use 1 and 0 for a binary mask, or between -1.0 and 1.0 for a varying mask
        for neighbourhood in range(0, self.initial_neighbourhood + 1):
            nsq = neighbourhood ** 2
            for i in range(self.nr_outputs):
                ix, iy = self.coords(i)
                for j in range(self.nr_outputs):
                    jx, jy = self.coords(j)
                    sqdist = (jx - ix) ** 2 + (jy - iy) ** 2
                    self.neighbourhood_lookup[neighbourhood, i, j] = 1 if sqdist <= nsq else 0

#     def get_weights(self, outputIndex):
#         return self.weights[:, outputIndex].tolist()

    def fit_transform(self, original_instances):

        # mask out instances containing NaNs and remove them
        instance_mask = ~np.any(np.isnan(original_instances), axis=1)
        nr_original_instances = original_instances.shape[0]
        valid_instances = original_instances[instance_mask, :]

        # randomly re-shuffle the remaining instances.
        # TODO consider reshuffling after every iteration
        instances = np.copy(valid_instances)
        #rng = np.random.default_rng(seed=self.seed)
        #rng.shuffle(instances)
        np.random.shuffle(instances)  # will change every time

        nr_inputs = instances.shape[1]
        nr_instances = instances.shape[0]

        weights = np.zeros((self.nr_outputs, nr_inputs))
        for output_idx in range(0, self.nr_outputs):
            weights[output_idx, :] = instances[self.rng.choice(range(0, nr_instances)), :]

#         p = Progress("SOM", silent=not self.verbose)
#         progress_frac = 0.0
#         p.report("Starting", progress_frac)

        for iteration in range(self.iters):
            # reduce the learning rate and neighbourhood size linearly as training progresses
            learn_rate = self.learn_rate_initial - (self.learn_rate_initial - self.learn_rate_final) * (
                        (iteration + 1) / self.iters)
            neighbour_limit = round(self.initial_neighbourhood * (1 - (iteration + 1) / self.iters))
            neighbourhood_mask = self.neighbourhood_lookup[neighbour_limit, :, :]
            batch_size = nr_instances if not minibatch_size else minibatch_size

            index = 0
            while index < nr_instances:
                last_index = min(index + batch_size, nr_instances)
                train_batch(instances[index:last_index, :], weights, learn_rate, neighbourhood_mask)
                index += batch_size

#             progress_frac = iteration / self.iters
#             p.report("Training neighbourhood=%d" % (neighbour_limit), progress_frac)

#         p.complete("SOM Training Complete")

        # compute final scores from the trained weights
        valid_scores = compute_scores(valid_instances, weights, self.gridwidth, self.minibatch_size)

        # restore the results into the same order as the input array
        scores = np.zeros(shape=(nr_original_instances, 2))
        scores[:, :] = np.nan
        scores[instance_mask, :] = valid_scores
        return scores

    def coords(self, output):
        return (output % self.gridwidth, output // self.gridwidth)

#     def get_output(self, x, y):
#         return x + (y * self.gridwidth)

In [15]:
# (using settings already run above)

In [16]:
s = SelfOrganisingMap(gridsize, gridsize, iters, seed=1, verbose=True, minibatch_size=1000)
%timeit scores = s.fit_transform(instances)

186 ms ± 384 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
s = SelfOrganisingMap(gridsize, gridsize, iters, seed=1, verbose=True, minibatch_size=None)
%timeit scores = s.fit_transform(instances)

185 ms ± 313 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Even faster than simple CuPy swap, though requires a little more effort and brings in differences from running on CPU. Simple replacement of NumPy with CuPy means you could run on GPU by changing 1 line. If it became important to reduce the timings further then using the reduction kernel would be useful.

This batch based approach is much better suited to running on a GPU.

# Updates...

In above examples the `np.add.at` commands were replaced due to there being no `cupy.add.at` available, e.g.
```python
np.add.at(numerator, weight_indices, updates)
```
replaced with
```python
numerator[weight_indices] = numerator[weight_indices] + updates
```
However `weight_indices` may contain repeated index values and this replacement code does not account for this (will only increment repeated elements once). So we should use an alternative to maintain the original functionality.

Cupy has the method `cupyx.scatter_add` which behaves exactly as `np.add.at` (https://docs.cupy.dev/en/stable/reference/generated/cupyx.scatter_add.html) so can use:
```python
cupyx.scatter_add(numerator, weight_indices, updates)
```

Another change we needed to make when using CuPy as drop in replacement for NumPy was to replace this:
```python
instances = np.copy(valid_instances)
rng = np.random.default_rng(seed=self.seed)
rng.shuffle(instances)
```

with this:
```python
instances = np.copy(valid_instances)
np.random.shuffle(instances)
```

because there was no `cupy.random.default_rng`.

At cupy version 9 and up, there is `cupy.random.default_rng`, however this generator object doesn't have the shuffle method. CuPy and the underlying libraries are constantly being updated so it is likely that this option will be added at some point!

In the meantime we can get around it with setting a random seed then using shuffling:
```python
instances = np.copy(valid_instances)
np.random.seed(seed=self.seed)
np.random.shuffle(instances)
```

In [18]:
import cupy as np
import cupyx

# def find_bmu(instances, weights):
#     print(instances.shape, weights.shape)
#     sqdiffs = (instances[:, :, None] - np.transpose(weights)) ** 2
#     sumsqdiffs = sqdiffs.sum(axis=1)
#     return sumsqdiffs.argmin(axis=1)

sqsum_kernel = np.ReductionKernel(
    'T x, T y',  # input params
    'T z',  # output params
    '(x - y) * (x - y)',  # map
    'a + b',  # reduce
    'z = a',  # post-reduction map
    '0',  # identity value
    'sqsum'  # kernel name
    )

def find_bmu(instances, weights):
    inarr = instances[:, :, None]
    sumsdiffs = sqsum_kernel(inarr, weights.T, axis=1)
    return np.argmin(sumsdiffs, axis=1)  


def train_batch(instances, weights, learn_rate, neighbourhood_lookup):
    # winners(#instances) holds the index of the closest weight for each instance
    winners = find_bmu(instances, weights)
    # now find the neighbours of each winner that are also activated by each instance
    # nhoods(#activations,2) holds the instance index and the weight index for each activation
    nwinners = neighbourhood_lookup[winners, :]
    nhoods = np.argwhere(nwinners)

    # get the indices
    weight_indices = nhoods[:, 1]
    instance_indices = nhoods[:, 0]
    fractions = nwinners[instance_indices, weight_indices]
    # print(fractions.shape,np.min(fractions),np.max(fractions))

    # get the updates
    updates = -learn_rate * fractions[:, None] * (weights[weight_indices, :] - instances[instance_indices])

    # aggregate the updates for each weight
    numerator = np.zeros(shape=weights.shape)
    cupyx.scatter_add(numerator, weight_indices, updates)
    denominator = np.zeros(shape=weights.shape[:1])[:, None]
    cupyx.scatter_add(denominator, weight_indices, 1)
    denominator = np.where(numerator == 0, 1, denominator)  # fix annoying divide by zero warning
    weight_updates = numerator / denominator

    # update the weights
    weights += weight_updates
    
def compute_scores(instances, weights, gridwidth, minibatch_size):
    index = 0
    nr_instances = instances.shape[0]
    batch_size = nr_instances if not minibatch_size else minibatch_size
    bmus = np.zeros(shape=(nr_instances,), dtype=int)
    while index < nr_instances:
        last_index = min(index + batch_size, nr_instances)
        bmus[index:last_index] = find_bmu(instances[index:last_index], weights)
        index += batch_size
    scores = np.vstack([bmus % gridwidth, bmus // gridwidth])
    return np.transpose(scores)

class SelfOrganisingMap(object):
    """
    Train Self Organising Map (SOM) with cells arranged in a 2-dimensional rectangular layout

    Parameters
    ----------
    iters : int
        the number of training iterations to use when training the SOM
    gridwidth : int
        number of cells across the grid
    gridheight : int
        number of cells down the grid
    initial_neighbourhood : int
        the initial neighbourhood size

    Keyword Parameters
    ------------------
    verbose : bool
        whether to print progress messages
    seed : int
        random seed - set to produce repeatable results
    """

    def __init__(self, gridwidth, gridheight, iters, initial_neighbourhood=None, verbose=False, seed=None,
                 minibatch_size=None):
        self.gridheight = gridheight
        self.gridwidth = gridwidth
        self.nr_outputs = self.gridwidth * self.gridheight
        self.iters = iters
        self.minibatch_size = minibatch_size

        self.initial_neighbourhood = initial_neighbourhood if initial_neighbourhood else int(gridsize / 3)
        self.verbose = verbose
        self.seed = seed
        self.rng = random.Random()
        if seed:
            self.rng.seed(seed)

        self.learn_rate_initial = 0.01
        self.learn_rate_final = 0.001
        self.neighbourhood_lookup = np.zeros(shape=(self.initial_neighbourhood + 1, self.nr_outputs, self.nr_outputs))

        # for each neighbourhood size 0,1,...initial_neighbourhood
        # build a lookup table where the fraction at neighbourhood_lookup[n,o1,o2]
        # indicates if (and how much) weight at index o2 is a neighbour of the weight at index o1 in neighbourhood size n
        # use 1 and 0 for a binary mask, or between -1.0 and 1.0 for a varying mask
        for neighbourhood in range(0, self.initial_neighbourhood + 1):
            nsq = neighbourhood ** 2
            for i in range(self.nr_outputs):
                ix, iy = self.coords(i)
                for j in range(self.nr_outputs):
                    jx, jy = self.coords(j)
                    sqdist = (jx - ix) ** 2 + (jy - iy) ** 2
                    self.neighbourhood_lookup[neighbourhood, i, j] = 1 if sqdist <= nsq else 0

#     def get_weights(self, outputIndex):
#         return self.weights[:, outputIndex].tolist()

    def fit_transform(self, original_instances):

        # mask out instances containing NaNs and remove them
        instance_mask = ~np.any(np.isnan(original_instances), axis=1)
        nr_original_instances = original_instances.shape[0]
        valid_instances = original_instances[instance_mask, :]

        # randomly re-shuffle the remaining instances.
        # TODO consider reshuffling after every iteration
        instances = np.copy(valid_instances)
        #rng = np.random.default_rng(seed=self.seed)
        #rng.shuffle(instances)
        np.random.seed(seed=self.seed)
        np.random.shuffle(instances)

        nr_inputs = instances.shape[1]
        nr_instances = instances.shape[0]

        weights = np.zeros((self.nr_outputs, nr_inputs))
        for output_idx in range(0, self.nr_outputs):
            weights[output_idx, :] = instances[self.rng.choice(range(0, nr_instances)), :]

#         p = Progress("SOM", silent=not self.verbose)
#         progress_frac = 0.0
#         p.report("Starting", progress_frac)

        for iteration in range(self.iters):
            # reduce the learning rate and neighbourhood size linearly as training progresses
            learn_rate = self.learn_rate_initial - (self.learn_rate_initial - self.learn_rate_final) * (
                        (iteration + 1) / self.iters)
            neighbour_limit = round(self.initial_neighbourhood * (1 - (iteration + 1) / self.iters))
            neighbourhood_mask = self.neighbourhood_lookup[neighbour_limit, :, :]
            batch_size = nr_instances if not minibatch_size else minibatch_size

            index = 0
            while index < nr_instances:
                last_index = min(index + batch_size, nr_instances)
                train_batch(instances[index:last_index, :], weights, learn_rate, neighbourhood_mask)
                index += batch_size

#             progress_frac = iteration / self.iters
#             p.report("Training neighbourhood=%d" % (neighbour_limit), progress_frac)

#         p.complete("SOM Training Complete")

        # compute final scores from the trained weights
        valid_scores = compute_scores(valid_instances, weights, self.gridwidth, self.minibatch_size)

        # restore the results into the same order as the input array
        scores = np.zeros(shape=(nr_original_instances, 2))
        scores[:, :] = np.nan
        scores[instance_mask, :] = valid_scores
        return scores

    def coords(self, output):
        return (output % self.gridwidth, output // self.gridwidth)

#     def get_output(self, x, y):
#         return x + (y * self.gridwidth)

In [19]:
s = SelfOrganisingMap(gridsize, gridsize, iters, seed=1, verbose=True, minibatch_size=1000)
%timeit scores = s.fit_transform(instances)

175 ms ± 498 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
s = SelfOrganisingMap(gridsize, gridsize, iters, seed=1, verbose=True, minibatch_size=None)
%timeit scores = s.fit_transform(instances)

174 ms ± 226 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


These changes do not hinder the timings, in fact they are very slightly faster, due to the `cupyx.scatter_add` change.

## Errors for larger gridsizes

We can continue to benefit from these increases in speed for larger grid sizes, but as we approach grid size of around 100 x 100, errors occur. For grids of around 200 x 200 we get a memory error, but for 100 x 100 we see a different error. I think this still may be a memory issue as the line it is complaining about shouldn't be a problem. Further investigation needed to confirm this.

In [21]:
s = SelfOrganisingMap(50, 50, iters, seed=1, verbose=True, minibatch_size=None)
%time scores = s.fit_transform(instances)

CPU times: user 990 ms, sys: 428 ms, total: 1.42 s
Wall time: 1.43 s


In [22]:
s = SelfOrganisingMap(100, 100, iters, seed=1, verbose=True, minibatch_size=None)
%time scores = s.fit_transform(instances)

CompileException: /opt/conda/lib/python3.8/site-packages/cupy/core/include/cupy/carray.cuh(185): error: the size of an array must be greater than zero
          detected during instantiation of class "CArray<T, _ndim, _c_contiguous, _use_32bit_indexing> [with T=double, _ndim=0, _c_contiguous=true, _use_32bit_indexing=false]" 
/tmp/tmpefcus7te/28e7b606c3dd6810485fbd9d6fdb1c5d_2.cubin.cu(8): here

/opt/conda/lib/python3.8/site-packages/cupy/core/include/cupy/carray.cuh(186): error: the size of an array must be greater than zero
          detected during instantiation of class "CArray<T, _ndim, _c_contiguous, _use_32bit_indexing> [with T=double, _ndim=0, _c_contiguous=true, _use_32bit_indexing=false]" 
/tmp/tmpefcus7te/28e7b606c3dd6810485fbd9d6fdb1c5d_2.cubin.cu(8): here

/opt/conda/lib/python3.8/site-packages/cupy/core/include/cupy/carray.cuh(202): error: the size of an array must be greater than zero
          detected during instantiation of class "CArray<T, _ndim, _c_contiguous, _use_

In [23]:
s = SelfOrganisingMap(200, 200, iters, seed=1, verbose=True, minibatch_size=None)
%time scores = s.fit_transform(instances)

OutOfMemoryError: Out of memory allocating 76,800,000,000 bytes (allocated so far: 10,831,452,672 bytes).